# Adding Features to Kickoff Data: Data Read in and Data Clean Pt. 2

In [1]:
import datetime as dt
import time

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth' ,999)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures

np.random.seed(3792)

In [2]:
# #Initial Datasets
# kickoff_df = pd.read_csv('../data/kickoff_plays.csv')
# punt_df = pd.read_csv('../data/punt_plays.csv')

In [3]:
#Data Read in
games_df   = pd.read_csv('../data/games.csv')
players_df = pd.read_csv('../data/players.csv')
track_2018 = pd.read_csv('../data/tracking2018.csv')
#track_2019 = pd.read_csv('../data/tracking2019.csv')
#track_2020 = pd.read_csv('../data/tracking2020.csv')

In [31]:
kickoffs_df = pd.read_csv('../data/kickoff_no_nulls.csv')
kickoffs_df.drop(columns ='Unnamed: 0', inplace=True)
kickoffs_df.head(1)

,gameId,playId,playDescription,quarter,possessionTeam,kickerId,returnerId,yardlineSide,yardlineNumber,gameClock,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,kickLength,kickReturnYardage,absoluteYardlineNumber,hangTime,kickType,kickDirectionIntended,kickDirectionActual,returnDirectionIntended,returnDirectionActual,missedTackler,assistTackler,tackler,kickoffReturnFormation,specialTeamsSafeties
0,2018090600,677,"M.Bosher kicks 64 yards from ATL 35 to PHI 1. S.Gibson to PHI 31 for 30 yards (D.Kazee, D.Riley).",1,ATL,37267.0,44979,ATL,35,05:01:00,NP,NP,0,0,3,64.0,30.0,75,4.06,D,R,R,C,C,ATL 83,ATL 22,ATL 27,8-0-2,ATL 17; ATL 22


## Adding Additional Data to Kickoff Data 

In [32]:
kickoffs_df.columns

Index(['gameId', 'playId', 'playDescription', 'quarter', 'possessionTeam',
       'kickerId', 'returnerId', 'yardlineSide', 'yardlineNumber', 'gameClock',
       'penaltyCodes', 'penaltyJerseyNumbers', 'penaltyYards',
       'preSnapHomeScore', 'preSnapVisitorScore', 'kickLength',
       'kickReturnYardage', 'absoluteYardlineNumber', 'hangTime', 'kickType',
       'kickDirectionIntended', 'kickDirectionActual',
       'returnDirectionIntended', 'returnDirectionActual', 'missedTackler',
       'assistTackler', 'tackler', 'kickoffReturnFormation',
       'specialTeamsSafeties'],
      dtype='object')

In [33]:
kickoffs_df = pd.merge(left=kickoffs_df, right=games_df)
kickoffs_df.head(1)

,gameId,playId,playDescription,quarter,possessionTeam,kickerId,returnerId,yardlineSide,yardlineNumber,gameClock,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,kickLength,kickReturnYardage,absoluteYardlineNumber,hangTime,kickType,kickDirectionIntended,kickDirectionActual,returnDirectionIntended,returnDirectionActual,missedTackler,assistTackler,tackler,kickoffReturnFormation,specialTeamsSafeties,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr
0,2018090600,677,"M.Bosher kicks 64 yards from ATL 35 to PHI 1. S.Gibson to PHI 31 for 30 yards (D.Kazee, D.Riley).",1,ATL,37267.0,44979,ATL,35,05:01:00,NP,NP,0,0,3,64.0,30.0,75,4.06,D,R,R,C,C,ATL 83,ATL 22,ATL 27,8-0-2,ATL 17; ATL 22,2018,1,09/06/2018,20:20:00,PHI,ATL


In [34]:
kickoffs_df.shape

(2642, 35)

In [35]:
kickoffs_df.columns

Index(['gameId', 'playId', 'playDescription', 'quarter', 'possessionTeam',
       'kickerId', 'returnerId', 'yardlineSide', 'yardlineNumber', 'gameClock',
       'penaltyCodes', 'penaltyJerseyNumbers', 'penaltyYards',
       'preSnapHomeScore', 'preSnapVisitorScore', 'kickLength',
       'kickReturnYardage', 'absoluteYardlineNumber', 'hangTime', 'kickType',
       'kickDirectionIntended', 'kickDirectionActual',
       'returnDirectionIntended', 'returnDirectionActual', 'missedTackler',
       'assistTackler', 'tackler', 'kickoffReturnFormation',
       'specialTeamsSafeties', 'season', 'week', 'gameDate', 'gameTimeEastern',
       'homeTeamAbbr', 'visitorTeamAbbr'],
      dtype='object')

In [36]:
### Adressing Date and Time Columns
#kickoffs_df['gameDate'] = pd.to_datetime(kickoffs_df['gameDate'])
#kickoffs_df.head(1)

#### Examining the season to season differences

In [37]:
kickoffs_df.groupby('season').mean()

,gameId,playId,quarter,kickerId,yardlineNumber,penaltyYards,preSnapHomeScore,preSnapVisitorScore,kickLength,kickReturnYardage,absoluteYardlineNumber,hangTime,week
season,,,,,,,,,,,,,
2018,2.018108e+09,1879.895388,2.383577,39911.353206,34.876265,0.744657,12.914511,10.794151,62.300337,23.505062,59.769404,3.820810,9.461192
2019,2.019109e+09,1878.793388,2.356553,42605.668241,34.811098,0.873672,11.136954,11.194805,62.454545,23.134593,59.569067,3.865065,9.685950
2020,2.020168e+09,1883.055188,2.409492,42782.947020,34.690949,0.507726,12.529801,11.966887,62.197572,22.684327,59.668874,3.774238,9.368653


In [38]:
kickoffs_df['season'].value_counts()

2020    906
2018    889
2019    847
Name: season, dtype: int64

### Subset to just 2018 Kick Returns
Given the large file sizes for each nfl season's tracking data (they each are around 1.5 - 1.75 GBs) I have made the decision to just examine the kick return tracking data for the 2018 season as it has the highest kick return average.

In [39]:
kickoffs_2018_df = kickoffs_df[kickoffs_df['season'] == 2018]
kickoffs_2018_df.head(1)

,gameId,playId,playDescription,quarter,possessionTeam,kickerId,returnerId,yardlineSide,yardlineNumber,gameClock,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,kickLength,kickReturnYardage,absoluteYardlineNumber,hangTime,kickType,kickDirectionIntended,kickDirectionActual,returnDirectionIntended,returnDirectionActual,missedTackler,assistTackler,tackler,kickoffReturnFormation,specialTeamsSafeties,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr
0,2018090600,677,"M.Bosher kicks 64 yards from ATL 35 to PHI 1. S.Gibson to PHI 31 for 30 yards (D.Kazee, D.Riley).",1,ATL,37267.0,44979,ATL,35,05:01:00,NP,NP,0,0,3,64.0,30.0,75,4.06,D,R,R,C,C,ATL 83,ATL 22,ATL 27,8-0-2,ATL 17; ATL 22,2018,1,09/06/2018,20:20:00,PHI,ATL


In [40]:
kickoffs_2018_df = pd.merge(left=kickoffs_2018_df, right=track_2018)
kickoffs_2018_df.head(1)

,gameId,playId,playDescription,quarter,possessionTeam,kickerId,returnerId,yardlineSide,yardlineNumber,gameClock,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,kickLength,kickReturnYardage,absoluteYardlineNumber,hangTime,kickType,kickDirectionIntended,kickDirectionActual,returnDirectionIntended,returnDirectionActual,missedTackler,assistTackler,tackler,kickoffReturnFormation,specialTeamsSafeties,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,playDirection
0,2018090600,677,"M.Bosher kicks 64 yards from ATL 35 to PHI 1. S.Gibson to PHI 31 for 30 yards (D.Kazee, D.Riley).",1,ATL,37267.0,44979,ATL,35,05:01:00,NP,NP,0,0,3,64.0,30.0,75,4.06,D,R,R,C,C,ATL 83,ATL 22,ATL 27,8-0-2,ATL 17; ATL 22,2018,1,09/06/2018,20:20:00,PHI,ATL,2018-09-07T01:34:29.300,79.22,26.19,4.47,1.58,0.44,289.13,305.37,None,37267.0,Matt Bosher,5.0,P,away,1,left


In [41]:
kickoffs_2018_df.columns

Index(['gameId', 'playId', 'playDescription', 'quarter', 'possessionTeam',
       'kickerId', 'returnerId', 'yardlineSide', 'yardlineNumber', 'gameClock',
       'penaltyCodes', 'penaltyJerseyNumbers', 'penaltyYards',
       'preSnapHomeScore', 'preSnapVisitorScore', 'kickLength',
       'kickReturnYardage', 'absoluteYardlineNumber', 'hangTime', 'kickType',
       'kickDirectionIntended', 'kickDirectionActual',
       'returnDirectionIntended', 'returnDirectionActual', 'missedTackler',
       'assistTackler', 'tackler', 'kickoffReturnFormation',
       'specialTeamsSafeties', 'season', 'week', 'gameDate', 'gameTimeEastern',
       'homeTeamAbbr', 'visitorTeamAbbr', 'time', 'x', 'y', 's', 'a', 'dis',
       'o', 'dir', 'event', 'nflId', 'displayName', 'jerseyNumber', 'position',
       'team', 'frameId', 'playDirection'],
      dtype='object')

In [42]:
kickoffs_2018_df['displayName'].value_counts()

football             97885
Nick Dzubnar          8591
Adrian Phillips       8492
Brandon Facyson       8174
Clayton Fejedelem     7859
                     ...  
Chester Rogers          84
Brett Kern              83
Amari Cooper            82
Ezekiel Elliott         82
Cole Beasley            82
Name: displayName, Length: 941, dtype: int64

In [43]:
kickoffs_2018_df.shape

(2251355, 51)

### 2018 Kick Returns Data Clean Part 2

In [44]:
#Lets check for nulls first; looks as though all of the nulls are from the football in the tracking data
kickoffs_2018_df.isna().sum().sort_values(ascending = False)[:10]

position                97885
jerseyNumber            97885
nflId                   97885
dir                     97885
o                       97885
gameId                      0
x                           0
specialTeamsSafeties        0
season                      0
week                        0
dtype: int64

In [45]:
#Lets check our assumption of nulls for football, yep it just for the football tracking
kickoffs_2018_df[kickoffs_2018_df['displayName'] != 'football'].isna().sum().sort_values(ascending = False)[:5]

gameId                  0
s                       0
specialTeamsSafeties    0
season                  0
week                    0
dtype: int64

In [46]:
kickoffs_2018_df['position'].value_counts()

CB     258959
WR     246857
TE     191706
RB     190282
LB     188034
OLB    175555
DB     148421
SS     140713
ILB    137922
FS     137792
K       82678
FB      69628
MLB     68281
DE      58748
S       16039
P       15392
HB      11604
QB      11459
G        2147
DL        958
DT        211
LS         84
Name: position, dtype: int64

In [47]:
#Lets fill in those nulls, FBL for football
kickoffs_2018_df['position'] = kickoffs_2018_df['position'].fillna('FBL')

In [48]:
#Should be able to use a jersey number of 0 for football
kickoffs_2018_df[kickoffs_2018_df['jerseyNumber'] == 0]

,gameId,playId,playDescription,quarter,possessionTeam,kickerId,returnerId,yardlineSide,yardlineNumber,gameClock,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,kickLength,kickReturnYardage,absoluteYardlineNumber,hangTime,kickType,kickDirectionIntended,kickDirectionActual,returnDirectionIntended,returnDirectionActual,missedTackler,assistTackler,tackler,kickoffReturnFormation,specialTeamsSafeties,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,playDirection


In [49]:
kickoffs_2018_df['jerseyNumber'] = kickoffs_2018_df['jerseyNumber'].fillna(0).astype(int)
kickoffs_2018_df[['jerseyNumber']]

,jerseyNumber
0,5
1,5
2,5
3,5
4,5
...,...
2251350,0
2251351,0
2251352,0
2251353,0


In [50]:
kickoffs_2018_df['nflId'].value_counts()

43062.0    8591
42116.0    8492
46729.0    8174
43534.0    7859
45020.0    7754
           ... 
39982.0      84
33338.0      83
38947.0      82
42347.0      82
43293.0      82
Name: nflId, Length: 943, dtype: int64

In [51]:
#Is there an nfl id of 11111? Turns out no, lets use that for football
kickoffs_2018_df[kickoffs_2018_df['nflId'] == 11111]

,gameId,playId,playDescription,quarter,possessionTeam,kickerId,returnerId,yardlineSide,yardlineNumber,gameClock,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,kickLength,kickReturnYardage,absoluteYardlineNumber,hangTime,kickType,kickDirectionIntended,kickDirectionActual,returnDirectionIntended,returnDirectionActual,missedTackler,assistTackler,tackler,kickoffReturnFormation,specialTeamsSafeties,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,playDirection


In [52]:
kickoffs_2018_df['nflId'] = kickoffs_2018_df['nflId'].fillna(11111).astype(int)
kickoffs_2018_df[['nflId']]

,nflId
0,37267
1,37267
2,37267
3,37267
4,37267
...,...
2251350,11111
2251351,11111
2251352,11111
2251353,11111


In [53]:
#Did I get all the nulls filled? not yet need direction and orientation, hmm how to fill these....
kickoffs_2018_df.isna().sum().sort_values(ascending = False)[:5]

dir                     97885
o                       97885
gameId                      0
y                           0
specialTeamsSafeties        0
dtype: int64

In [54]:
kickoffs_2018_df['dir'].value_counts()

90.90     191
90.73     187
90.12     187
271.74    186
270.05    186
         ... 
170.39     20
173.22     19
193.26     19
160.79     17
17.08      17
Name: dir, Length: 36001, dtype: int64

In [55]:
#Can't even put it as 0... I guess I'll just need to remember to not use the data ith football when I'm looking at direction or orientation
kickoffs_2018_df[kickoffs_2018_df['dir'] == 0].head()

,gameId,playId,playDescription,quarter,possessionTeam,kickerId,returnerId,yardlineSide,yardlineNumber,gameClock,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,kickLength,kickReturnYardage,absoluteYardlineNumber,hangTime,kickType,kickDirectionIntended,kickDirectionActual,returnDirectionIntended,returnDirectionActual,missedTackler,assistTackler,tackler,kickoffReturnFormation,specialTeamsSafeties,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,playDirection
75334,2018090905,3524,S.Gostkowski kicks 66 yards from NE 35 to HOU -1. T.Ervin to HOU 31 for 32 yards (M.Slater; G.Grissom).,4,NE,30932.0,43408,NE,35,09:48:00,NP,NP,0,27,13,66.0,32.0,75,4.10,D,C,C,R,R,returner_tackled,NE 96,NE 18,8-0-2,NE 23; NE 32,2018,1,09/09/2018,13:00:00,NE,HOU,2018-09-09T19:37:56.300,9.13,26.49,3.08,0.83,0.31,75.67,0.0,None,43408,Tyler Ervin,21,RB,away,43,left
89922,2018090907,1867,"A.Rosas kicks 65 yards from NYG 35 to JAX 0. J.Mickens to JAX 27 for 27 yards (A.Hamilton, R.Armstrong).",2,NYG,43937.0,44029,NYG,35,00:04:00,NP,NP,0,6,13,65.0,27.0,75,3.34,D,R,R,L,L,returner_tackled,NYG 55,NYG 30,8-0-2,NYG 24; NYG 36,2018,1,09/09/2018,13:00:00,NYG,JAX,2018-09-09T18:19:12.400,35.47,41.31,1.82,3.95,0.16,92.22,0.0,None,44961,Blair Brown,53,OLB,away,78,left
261400,2018092303,637,A.Rosas kicks 65 yards from NYG 35 to HOU 0. T.Ervin pushed ob at HOU 22 for 22 yards (L.Carter; K.Wynn).,1,NYG,43937.0,43408,NYG,35,04:10:00,NP,NP,0,3,7,65.0,22.0,75,4.33,D,R,R,L,L,returner_tackled,Solo,NYG 59,8-0-2,NYG 20; NYG 21,2018,3,09/23/2018,13:00:00,HOU,NYG,2018-09-23T17:25:34.500,31.48,36.84,4.17,3.34,0.40,79.26,0.0,None,46778,A.J. Moore,33,DB,home,80,left
424986,2018093008,1101,P.Dawson kicks 62 yards from ARI 35 to SEA 3. R.Penny to SEA 26 for 23 yards (D.Gardeck).,2,ARI,23860.0,46096,ARI,35,14:04:00,NP,NP,0,3,7,62.0,23.0,75,4.04,D,C,C,L,L,returner_tackled,Solo,ARI 92,8-0-2,ARI 28; ARI 36,2018,4,09/30/2018,16:05:00,ARI,SEA,2018-09-30T20:52:19.100,37.69,19.43,5.46,3.02,0.55,320.43,0.0,None,40002,Jamar Taylor,28,CB,home,79,left
475410,2018093012,4226,J.Tucker kicks 70 yards from BAL 35 to PIT -5. R.Switzer to PIT 16 for 21 yards (C.Board).,4,BAL,39470.0,44946,BAL,35,01:56:00,NP,NP,0,14,26,70.0,21.0,75,4.08,D,C,C,C,C,returner_tackled,Solo,BAL 49,8-0-2,BAL 25; BAL 27,2018,4,09/30/2018,20:20:00,PIT,BAL,2018-10-01T03:25:57.700,29.87,19.43,2.86,1.10,0.28,160.19,0.0,None,38883,L.J. Fort,54,ILB,home,86,left


In [56]:
kickoffs_2018_df.dtypes

gameId                       int64
playId                       int64
playDescription             object
quarter                      int64
possessionTeam              object
kickerId                   float64
returnerId                  object
yardlineSide                object
yardlineNumber               int64
gameClock                   object
penaltyCodes                object
penaltyJerseyNumbers        object
penaltyYards                 int64
preSnapHomeScore             int64
preSnapVisitorScore          int64
kickLength                 float64
kickReturnYardage          float64
absoluteYardlineNumber       int64
hangTime                   float64
kickType                    object
kickDirectionIntended       object
kickDirectionActual         object
returnDirectionIntended     object
returnDirectionActual       object
missedTackler               object
assistTackler               object
tackler                     object
kickoffReturnFormation      object
specialTeamsSafeties

In [58]:
#Saving current dataframe with football and no players
kickoffs_2018_df.to_csv('../data/kickoffs_2018_fb_no_players.csv', index=False)

In [59]:
#Since we've seen some issues with the tracking on football we can create a second dataframe with the player data, but not football
%time

kickoffs_2018_df = pd.merge(left=kickoffs_2018_df, right=players_df)

Wall time: 0 ns


In [60]:
#Hurray no nulls!
kickoffs_2018_df.isna().sum().sort_values(ascending = False)[:10]

gameId             0
playId             0
week               0
gameDate           0
gameTimeEastern    0
homeTeamAbbr       0
visitorTeamAbbr    0
time               0
x                  0
y                  0
dtype: int64

In [62]:
#Creating 2018 Kickoffs with players, tracking, but not football
kickoffs_2018_df.to_csv('../data/kickoffs_2018_w_players.csv', index=False)